In [17]:
# Import required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
import json
from pathlib import Path
from sklearn.linear_model import LinearRegression


# %%
# Load the data into a Pandas DataFrame
stock_df = pd.read_csv(
    "../Resources/data.csv",
    index_col="Company")

# %%
# Display sample data
stock_df.head(10)

# %%
# Remove the $ sign from the columns 
stock_df = stock_df.replace({'\$': ' '}, regex=True)

# Review the changes 
stock_df.head(5)

# %%
stock_df.info()

# %%
# Check for missing values
null_counts = stock_df.isnull().sum()
print(null_counts)

# %%
# Convert to datetime
stock_df['Date'] = pd.to_datetime(stock_df['Date'])

# Find the oldest and newest dates
oldest_date = stock_df['Date'].min()
newest_date = stock_df['Date'].max()

print(oldest_date)
print(newest_date)

# %%
# Sort the df by company and date 
stock_df.sort_values(by=['Company', 'Date'], inplace=True)
stock_df.head(5)

# %%
# Calculating the change between closing and opening prices per company
stock_df['Close/Last'] = pd.to_numeric(stock_df['Close/Last'], errors='coerce')
stock_df['Open'] = pd.to_numeric(stock_df['Open'], errors='coerce')

stock_df['Change'] = stock_df.groupby('Company')['Close/Last'].shift(1) - stock_df['Open']
stock_df.head(5)

# %%
# Delete null value in Change column
stock_df.dropna(subset=['Change'], inplace=True)
stock_df.head(5)

# %%
# Filter for AAPL data only
appl_data = stock_df[stock_df.index == 'AAPL']

appl_data.head()


<class 'pandas.core.frame.DataFrame'>
Index: 25160 entries, AAPL to NFLX
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Date        25160 non-null  object
 1   Close/Last  25160 non-null  object
 2   Volume      25160 non-null  int64 
 3   Open        25160 non-null  object
 4   High        25160 non-null  object
 5   Low         25160 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.3+ MB
Date          0
Close/Last    0
Volume        0
Open          0
High          0
Low           0
dtype: int64
2013-07-18 00:00:00
2023-07-17 00:00:00


,Date,Close/Last,Volume,Open,High,Low,Change
Company,,,,,,,
AAPL,2013-07-19,15.1768,268548901,15.4679,15.4993,15.1554,-0.0480
AAPL,2013-07-22,15.2254,207648981,15.3379,15.3482,15.1953,-0.1611
AAPL,2013-07-23,14.9639,354477618,15.2143,15.2486,14.9539,0.0111
AAPL,2013-07-24,15.7325,591624923,15.6761,15.8782,15.545,-0.7122
AAPL,2013-07-25,15.6607,229432412,15.7393,15.7643,15.5646,-0.0068


In [5]:

# %%
# Plot AAPL data
appl_plot = appl_data.hvplot.scatter(
    x="Open",
    y="Close/Last",
    title="AAPL Open vs Close prices"
)
appl_plot


:Scatter   [Open]   (Close/Last)

In [16]:

# %%
# Reformat data of the independent variable X as a single-column array
X = appl_data["Open"].values.reshape(-1, 1)

# Display sample data
X[:5]

# %%
# The shape of X is 30 samples, with a single feature (column)
X.shape

# %%
# Create an array for the dependent variable y
y = appl_data["Close/Last"]

# %%
# Create a model with scikit-learn
model = LinearRegression()

# %%
# Fit the data into the model
model.fit(X, y)


LinearRegression()

In [15]:

# %%
# Display the slope
print(f"Model's slope: {model.coef_}")


Model's slope: [1.00088919]


In [10]:

# %%
# Display the y-intercept
print(f"Model's y-intercept: {model.intercept_}")


Model's y-intercept: -0.001175747208904454


In [11]:

# %%
# Display the model's best fit line formula
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]}X")


Model's formula: y = -0.001175747208904454 + 1.0008891851424842X


In [12]:

# %%
# Display the formula to predict the close price if the open price is $30
print(f"Model's formula: y = {model.intercept_} + {model.coef_[0]} * 30")


Model's formula: y = -0.001175747208904454 + 1.0008891851424842 * 30


In [13]:

# predict the close price if the open price is $30
y_30 = model.intercept_ + model.coef_[0] * 30

# Display the prediction
print(f"Predicted stock closing price if the open price is $30: ${y_30:.2f}")


Predicted stock closing price after 30 days: $30.03


In [22]:

# %%
# Make predictions using the X set
predicted_y_values = model.predict(X)

# %%
# Create a copy of the original data
df_close_predicted = appl_data.copy()

# Add a column with the predicted close values
df_close_predicted["close_predicted"] = predicted_y_values

# Display sample data
df_close_predicted.head()

# %%
# Create a line plot of the predicted close values
best_fit_line = df_close_predicted.hvplot.line(
    x = "Open",
    y = "close_predicted",
    color = "red"
)
best_fit_line


:Curve   [Open]   (close_predicted)

In [23]:

# %%
# Superpose the original data and the best fit line
appl_plot * best_fit_line


:Overlay
   .Scatter.I :Scatter   [Open]   (Close/Last)
   .Curve.I   :Curve   [Open]   (close_predicted)

In [8]:

# %%
# Import relevant metrics - score, r2, mse, rmse, std - from Scikit-learn
from sklearn.metrics import mean_squared_error, r2_score

# %%
# Compute the metrics for the linear regression model
score = model.score(X, y, sample_weight=None)
r2 = r2_score(y, predicted_y_values)
mse = mean_squared_error(y, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is 0.9993356753460328.
The r2 is 0.9993356753460328.
The mean squared error is 1.82745464790373.
The root mean squared error is 1.3518338092767652.
The standard deviation is 52.44850565422048.


In [21]:
models_info = {}
# Save model information
model_info = {
    "intercept": model.intercept_,
    "coef": model.coef_[0]
}
# Store the model information in the dictionary
models_info['AAPL'] = model_info
# Save the model information to a JSON file named after the ticker symbol
filename = f"{'AAPL'}_model_info.json"
with open(filename, "w") as json_file:
    json.dump(model_info, json_file)